In [1]:
import dropbox
import pandas as pd
import time
import datetime
import os
from PIL import Image
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import *
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
import datetime
from email.mime.base import MIMEBase
import email.encoders as Encoders


def files():
    files = [content['path'].split('/')[-1]
             for content in metadata['contents'] if content['is_dir'] == False]
    size = [round(content['bytes'] / 10**6, 2)
            for content in metadata['contents'] if content['is_dir'] == False]
    dates = [content['modified'] for content in metadata['contents'] if content['is_dir'] == False]
    name = [i[0:i.find('_')].lower().title() for i in files]
    files = pd.DataFrame({'file_name': files, 'c_name': name, 'size': size, 'dates': dates})
    files['date_mod_d'] = [time.strptime(i, "%a, %d %b %Y %H:%M:%S %z") for i in dates]
    files['date_mod_d'] = [datetime.datetime(
        *files.date_mod_d.loc[i][:6]) - datetime.timedelta(hours=4) for i in files.index]
    files['date_mod'] = [files.loc[i].date_mod_d.date() for i in files.index]
    files['hour_mod'] = [files.loc[i].date_mod_d.time() for i in files.index]
    #tday = datetime.datetime.today().date()
    #files = files[files.date_mod == tday]
    del files['date_mod_d'], files['dates']
    return files

def watermark(x):
    for i in x:
        if 'AVI' or 'MPG' in i:
            clip = VideoFileClip(i)
            #logo = (logo.set_duration(clip.duration)
             #       .resize(height=80)
              #      .margin(right=8, top=8, opacity=0)  # (optional) logo-border padding
               #     .set_pos(("right", "bottom")))
            # logo = (ImageClip("pancita4.png")
            #.set_duration(clip.duration)
           #   .resize(height=80)
            #  .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
            # .set_pos(("right","bottom")))
            #final = CompositeVideoClip([clip, logo])
            final.write_videofile("Pancitas " + i + ".mp4")
        if 'JPG' in i:
            #plogo = Image.open("pancita4.png")
            #plogow, plogoh = plogo.size
            #plogo = plogo.resize((int(plogow*.1), int(plogoh*.1)))
            #plogow, plogoh = plogo.size
            pic = Image.open(i)
            picw, pich = pic.size
            pic.paste(plogo, (picw - plogow, pich - plogoh), plogo)
            pic.save('Pancitas ' + i)

        else:
            pass


def dl_files(x):
    client = dropbox.client.DropboxClient(
        'IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    df = pd.DataFrame(x.groupby('Nombre').Nombre.count())
    files = x.file_name
    os.chdir('.\\Clients')
    for i in df.index:
        c_dir = 'Pancitas Ultrasound {}'.format(i.lower().title())
        os.mkdir(c_dir)
        os.chdir('.\\' + c_dir)
        dl_df = []
        for k in files:
            if i.upper() in k:
                dl_df.append(k)
        for j in dl_df:
            f, metadata = client.get_file_and_metadata('/' + j)
            out = open(j, 'wb')
            out.write(f.read())
            out.close()
        watermark(dl_df)
        for l in os.listdir(os.getcwd()):
            if 'Pancitas' in l:
                pass
            else:
                os.remove(l)
        os.chdir('..\\')
    
    return
        
def send_mail():
    send = []
    for i in clients.index:
        for j in next(os.walk('.'))[1]:
            if clients.Nombre.loc[i] in j:
                c_name = clients.Nombre.loc[i]
                c_email = clients.Email.loc[i]
                folder = j
                msg = MIMEMultipart()
                msg['Subject'] = 'Ultrasound Imagery'
                msg['From'] = 'fapb88ve@gmail.com'
                msg['To'] = c_email
                txt = """Hello {}! 
                \nThese are the pictures from your session.
                \nShare your experience and enjoy a 15% discount on your next visit. 
                \nClick here to reedem: https://goo.gl/rC3z9H
            
                \nWish you all the best! Come back soon :)
 
                \nThanks
                \nPancita’s Team
                \nwww.pancitas.com
                \n(305) 542-9931"""
                text = MIMEText(txt.format(c_name))
                
                msg.attach(text)
                os.chdir('.\\'+folder)
                files = os.listdir(os.getcwd())
                for l in files:
                    if 'mp4' in l:
                        part = MIMEBase('application', "octet-stream")
                        fo=open(l,"rb")
                        part.set_payload(fo.read() )
                        Encoders.encode_base64(part)
                        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(l))
                        msg.attach(part)
                    if 'jpg' in l:
                        img_data = open(l, 'rb').read()
                        image = MIMEImage(img_data, name=os.path.basename(l))
                        msg.attach(image)
                s = smtplib.SMTP('smtp.gmail.com', 587)
                s.ehlo()
                s.starttls()
                s.login('fapb88ve@gmail.com', 'rabbitrun88ve')
                s.sendmail('fapb88ve@gmail.com', c_email, msg.as_string())
                s.quit()
                os.chdir('..')

In [2]:
def full():
    global sup
    #sup = input()
    os.chdir('.\\Logo')
    global logo, plogo, plogow, plogoh, client, metadata
    client = dropbox.client.DropboxClient(
        'IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    metadata = client.metadata('/')
    logo = ImageClip("pancita4.png")
    plogo = Image.open("pancita4.png")
    plogow, plogoh = plogo.size
    plogo = plogo.resize((int(plogow * .1), int(plogoh * .1)))
    plogow, plogoh = plogo.size
    os.chdir('..')
    # Time Function
    d_files = files()
    global clients
    clients = pd.read_excel('Registro de Clientes.xlsx')
    send_f = pd.merge(d_files, clients, left_on='c_name', right_on='Nombre',
                      how='inner')
    dl_files(send_f)
    send_mail()

if os.getcwd() != 'C:\\Users\\Frank Pinto\\desktop\\pds\\Pancitas\\Code':
    os.chdir('C:\\Users\\Frank Pinto\\desktop\\pds\\Pancitas\\Code')
full()
print(os.getcwd())
#os.chdir('..')

NameError: name 'final' is not defined

## Ramdon things

In [20]:
if os.getcwd() != 'C:\\Users\\Frank Pinto\\desktop\\pds\\Pancitas\\Code':
    os.chdir('C:\\Users\\Frank Pinto\\desktop\\pds\\Pancitas\\Code')
clients = pd.read_excel('Registro de Clientes.xlsx')

send = pd.DataFrame()
os.chdir('.\\Clients')
for i in clients.index:
    for j in next(os.walk('.'))[1]:
        if clients.Nombre.loc[i] in j:
            send.append({'email':clients.Email.loc[i], 'directory':j}, ignore_index=True)
            #send.append([clients.Email.loc[i], j])
            
                                
send



""


In [2]:
os.getcwd()

'C:\\Users\\Frank Pinto\\desktop\\pds\\Pancitas\\Code'